In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/reduced-dataset/annotations/meta.csv
/kaggle/input/reduced-dataset/annotations/val/95ca9e53-6469-4a7b-b7d7-37a8ccea308a.png
/kaggle/input/reduced-dataset/annotations/val/bcf1d7bd-94bd-49ad-9d83-69616408423c.png
/kaggle/input/reduced-dataset/annotations/val/58e16fa1-b685-4906-86f5-8d106384a688.png
/kaggle/input/reduced-dataset/annotations/val/c40269bb-332e-4192-975d-62382d4a3733.png
/kaggle/input/reduced-dataset/annotations/val/e9b20a37-162a-440c-93e6-aab51ba8fc79.png
/kaggle/input/reduced-dataset/annotations/val/47e671fd-10f3-49cc-8398-ce190bbe760a.png
/kaggle/input/reduced-dataset/annotations/val/e7b2acf0-03c8-4a57-9908-f0f664a5a0b6.png
/kaggle/input/reduced-dataset/annotations/val/ef986f59-ebb2-4364-860d-71cc271347dc.png
/kaggle/input/reduced-dataset/annotations/val/3b436ad8-b4ac-4069-99bc-373d43c056e4.png
/kaggle/input/reduced-dataset/annotations/val/eec6c232-9d61-411a-8b80-045c8e26e368.png
/kaggle/input/reduced-dataset/annotations/val/321a1383-283f-4914-b33e-dd0744cc9

In [2]:
import tensorflow as tf
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

IMG_SIZE = 256 # Resize to 256x256
NUM_CLASSES = 8  # Found from unique mask values

# Paths

# Function to load image
def load_image(image_file):
    img = cv2.imread(image_file)  # Read image (BGR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize
    img = img / 255.0  # Normalize (0 to 1)
    return img

# Function to load mask
def load_mask(mask_file):
    mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
    mask = cv2.resize(mask, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)  # Resize
    mask = np.expand_dims(mask, axis=-1)  # Add channel dimension
    return mask


In [3]:
from tensorflow.keras.utils import to_categorical

def process_mask(mask):
    mask = to_categorical(mask, num_classes=NUM_CLASSES)  # One-hot encode
    return mask


In [4]:
import os

# Define dataset paths
base_path = "/kaggle/input/reduced-dataset"
train_image_path = os.path.join(base_path, "images/train")
val_image_path = os.path.join(base_path, "images/val")
test_image_path = os.path.join(base_path, "images/test")

train_mask_path = os.path.join(base_path, "annotations/train")
val_mask_path = os.path.join(base_path, "annotations/val")
test_mask_path = os.path.join(base_path, "annotations/test")

# Function to get sorted file paths
def get_file_paths(image_path, mask_path):
    image_files = sorted([os.path.join(image_path, f) for f in os.listdir(image_path) if f.endswith(".jpg")])
    mask_files = sorted([os.path.join(mask_path, f) for f in os.listdir(mask_path) if f.endswith(".png")])
    return image_files, mask_files

# Get file paths
train_image_files, train_mask_files = get_file_paths(train_image_path, train_mask_path)
val_image_files, val_mask_files = get_file_paths(val_image_path, val_mask_path)
test_image_files, test_mask_files = get_file_paths(test_image_path, test_mask_path)

# Print counts to verify
print(f"Train Images: {len(train_image_files)}, Masks: {len(train_mask_files)}")
print(f"Val Images: {len(val_image_files)}, Masks: {len(val_mask_files)}")
print(f"Test Images: {len(test_image_files)}, Masks: {len(test_mask_files)}")


Train Images: 3500, Masks: 3500
Val Images: 500, Masks: 500
Test Images: 1000, Masks: 1000


In [5]:
import tensorflow as tf
import cv2
import numpy as np

IMG_SIZE = 256  # Resize all images to (256, 256)

def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize to uniform size
    img = img / 255.0  # Normalize to [0,1]
    return img

def load_mask(mask_path):
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)  # Grayscale mask
    mask = tf.image.resize(mask, (IMG_SIZE, IMG_SIZE), method='nearest')  # Keep labels intact
    mask = tf.cast(mask, tf.uint8)  # Ensure integer values
    return mask

def tf_load_data(image_path, mask_path):
    img = load_image(image_path)
    mask = load_mask(mask_path)
    return img, mask


In [6]:
BATCH_SIZE = 8

def create_dataset(image_files, mask_files):
    dataset = tf.data.Dataset.from_tensor_slices((image_files, mask_files))
    dataset = dataset.map(tf_load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE).shuffle(100).prefetch(tf.data.AUTOTUNE)
    return dataset

# Create datasets
train_dataset = create_dataset(train_image_files, train_mask_files)
val_dataset = create_dataset(val_image_files, val_mask_files)
test_dataset = create_dataset(test_image_files, test_mask_files)

# Check dataset output
for img, mask in train_dataset.take(1):
    print("Train Image shape:", img.shape)  # (batch, 256, 256, 3)
    print("Train Mask shape:", mask.shape)  # (batch, 256, 256, 8)


Train Image shape: (8, 256, 256, 3)
Train Mask shape: (8, 256, 256, 1)


In [7]:
import os
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import matplotlib.pyplot as plt


In [8]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Dropout, MaxPooling2D, concatenate
from tensorflow.keras.models import Model

def unet_resnet50(input_shape=(256, 256, 3), num_classes=8):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

    # Encoder layers
    c1 = base_model.get_layer('conv1_relu').output
    c2 = base_model.get_layer('conv2_block3_out').output
    c3 = base_model.get_layer('conv3_block4_out').output
    c4 = base_model.get_layer('conv4_block6_out').output
    bridge = base_model.get_layer('conv5_block3_out').output

    # Decoder
    d1 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(bridge)
    d1 = concatenate([d1, c4])
    d1 = Conv2D(512, (3, 3), activation='relu', padding='same')(d1)
    d1 = Conv2D(512, (3, 3), activation='relu', padding='same')(d1)

    d2 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(d1)
    d2 = concatenate([d2, c3])
    d2 = Conv2D(256, (3, 3), activation='relu', padding='same')(d2)
    d2 = Conv2D(256, (3, 3), activation='relu', padding='same')(d2)

    d3 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(d2)
    d3 = concatenate([d3, c2])
    d3 = Conv2D(128, (3, 3), activation='relu', padding='same')(d3)
    d3 = Conv2D(128, (3, 3), activation='relu', padding='same')(d3)

    d4 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(d3)
    d4 = concatenate([d4, c1])
    d4 = Conv2D(64, (3, 3), activation='relu', padding='same')(d4)
    d4 = Conv2D(64, (3, 3), activation='relu', padding='same')(d4)

    # Output Layer
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(d4)

    # Model
    model = Model(inputs=base_model.input, outputs=outputs)

    return model

# Define model
model = unet_resnet50()
model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 262, 262, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 128, 128, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 128, 128, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 128, 128, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 130, 130, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 64, 64, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 64, 64, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 64, 64, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 64, 64, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 64, 64, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 64, 64, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 64, 64, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 64, 64, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 64, 64, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 40,970,440 (156.29 MB)

 Trainable params: 40,917,320 (156.09 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, UpSampling2D

# Example: Adjusting the final layers of a U-Net-like model
def modify_model(model):
    x = model.output  # Get the original output

    # Upsample back to (256, 256)
    x = UpSampling2D(size=(2, 2), interpolation='bilinear')(x)  # (128 -> 256)

    # Ensure the final output has the correct shape
    x = Conv2D(1, (1, 1), activation="sigmoid")(x)  # 1-channel mask for binary segmentation

    # Create new model
    new_model = tf.keras.models.Model(inputs=model.input, outputs=x)
    return new_model

# Apply the modification
model = modify_model(model)

# Compile again
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])


In [10]:
# Callbacks
# Callbacks
checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Training
history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=3,
                    callbacks=[checkpoint, early_stopping])


Epoch 1/3
438/438 ━━━━━━━━━━━━━━━━━━━━ 241s 361ms/step - accuracy: 0.7737 - loss: 0.3829 - val_accuracy: 0.5301 - val_loss: 0.4952
Epoch 2/3
438/438 ━━━━━━━━━━━━━━━━━━━━ 117s 244ms/step - accuracy: 0.8387 - loss: 0.1553 - val_accuracy: 0.8115 - val_loss: 0.0836
Epoch 3/3
438/438 ━━━━━━━━━━━━━━━━━━━━ 117s 245ms/step - accuracy: 0.8493 - loss: -0.0012 - val_accuracy: 0.8359 - val_loss: -0.0989


In [11]:
import os
import cv2
import numpy as np
import pandas as pd

import numpy as np

import numpy as np
from itertools import chain

def rle_encode(mask):
    """
    Encodes a binary mask (numpy array of 0s and 1s) into a space-delimited RLE string.
    Pixels are read in Fortran order (top-to-bottom, then left-to-right).
    Each pair of numbers in the output represents the starting pixel (1-indexed) and 
    the run length.
    
    Example: '1 3 10 5' means pixels 1,2,3 and 10,11,12,13,14 are foreground.
    
    Args:
        mask (np.array): 2D binary mask.
        
    Returns:
        str: RLE-encoded string.
    """
    # Flatten in Fortran order (column-major: top-to-bottom, then left-to-right)
    pixels = mask.flatten(order='F')
    # Find indices where mask is 1
    indices = np.where(pixels == 1)[0]
    if len(indices) == 0:
        return ''
    
    runs = []
    run_start = indices[0]
    run_length = 1
    # Loop over subsequent indices
    for i in range(1, len(indices)):
        if indices[i] == indices[i-1] + 1:
            run_length += 1
        else:
            # Append start and length (convert to 1-indexed)
            runs.extend([run_start + 1, run_length])
            run_start = indices[i]
            run_length = 1
    runs.extend([run_start + 1, run_length])
    # Convert list of numbers to space-delimited string
    return ' '.join(map(str, runs))


    
    total = shape[0]
    rles = []
    
    for i in range(total):
        img = masks[i, 0]
        rle = run_length_enc(img, top_n=top_n)
        rles.append(rle)
    
    return rles  # Return list of RLEs for submission
  # Return empty space if no runs (to avoid NaN errors)



# Define the path to the annotations folder for the test set
annotations_test_path = "/kaggle/input/reduced-dataset/annotations/test"

# List all mask files (assuming they are PNG images)
mask_files = sorted([f for f in os.listdir(annotations_test_path) if f.lower().endswith(".png")])
print("Total test masks found:", len(mask_files))

solution_data = []

for mask_file in mask_files:
    # Use the filename (without extension) as the unique id
    image_id = os.path.splitext(mask_file)[0]
    mask_path = os.path.join(annotations_test_path, mask_file)
    
    # Read the mask in grayscale
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"Warning: Could not read mask file {mask_path}")
        continue
    
    # Ensure the mask is binary (if needed, threshold it)
    # For example, assuming foreground > 0, convert to 0/1:
    mask = (mask > 0).astype(np.uint8)
    
    # Encode the binary mask using RLE
    rle_str = rle_encode(mask)
    
    solution_data.append({
        "id": image_id,
        "ground_truth": rle_str
    })

# Create a DataFrame from the solution data and save as CSV
solution_df = pd.DataFrame(solution_data)
solution_csv_path = "solutionf.csv"
solution_df.to_csv(solution_csv_path, index=False)

print("Solution CSV created successfully with", len(solution_df), "records.")
print("CSV saved at:", solution_csv_path)


Total test masks found: 1000
Solution CSV created successfully with 1000 records.
CSV saved at: solutionf.csv


In [12]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf

# --------------------- Run-Length Encoding (Updated) ---------------------
def encode_mask_to_rle(mask):
    '''
    mask: numpy array binary mask 
    1 - mask 
    0 - background
    Returns encoded run length 
    '''
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# --------------------- Create Submission CSV (Without "Usage" Column) ---------------------
def create_submission_file(masks, shape, index, submission_name="submission.csv"):
    """Creates the submission CSV file using predicted masks (NO "Usage" column)."""
    total = shape[0]
    rles = [encode_mask_to_rle(masks[i, 0]) for i in range(total)]

    # Write CSV file
    with open(submission_name, 'w+') as f:
        f.write("img,pixels\n")
        for i in range(total):
            f.write(f"{index[i]},{rles[i]}\n")
    print(f"✅ Submission file '{submission_name}' created successfully!")

# --------------------- Create Solution CSV (WITH Ground Truth & "Usage" Column) ---------------------
def create_solution_file(gt_masks, shape, index, solution_name="solution.csv"):
    """Creates the solution CSV file using ground truth masks with a 'Usage' column set to 'Public'."""
    total = shape[0]
    rles = [encode_mask_to_rle(gt_masks[i, 0]) for i in range(total)]

    # Create DataFrame with 'Usage' column
    df_solution = pd.DataFrame({"img": index, "pixels": rles, "Usage": "Public"})
    df_solution.to_csv(solution_name, index=False)
    print(f"✅ Solution file '{solution_name}' created successfully!")

# --------------------- Load Model ---------------------
model_path = "best_model.keras"
model = tf.keras.models.load_model(model_path)

# Load test images
test_images_path = "/kaggle/input/reduced-dataset/images/test"
test_image_files = sorted([f for f in os.listdir(test_images_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

image_ids = []
predicted_masks = []
ground_truth_masks = []

# Load ground truth masks
gt_masks_path = "/kaggle/input/reduced-dataset/masks/test"  # Adjust if needed

for image_file in test_image_files:
    image_id = os.path.splitext(image_file)[0]
    image_path = os.path.join(test_images_path, image_file)
    gt_mask_path = os.path.join(gt_masks_path, image_id + ".png")  # Adjust extension if needed
    
    # Load and preprocess test image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (256, 256))  # Resize for model
    image = image / 255.0  # Normalize
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    
    # Predict mask
    pred_mask = model.predict(image)[0]  # Remove batch dimension
    binary_mask = (pred_mask > 0.5).astype(np.uint8)  # Convert to binary mask
    
    # Resize mask back to original image shape
    original_image = cv2.imread(image_path)  # Load original for size reference
    orig_h, orig_w = original_image.shape[:2]
    binary_mask = cv2.resize(binary_mask, (orig_w, orig_h), interpolation=cv2.INTER_NEAREST)
    
    # Load ground truth mask
    gt_mask = cv2.imread(gt_mask_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
    gt_mask = cv2.resize(gt_mask, (orig_w, orig_h), interpolation=cv2.INTER_NEAREST)
    gt_mask = (gt_mask > 127).astype(np.uint8)  # Convert to binary mask

    predicted_masks.append(binary_mask)
    ground_truth_masks.append(gt_mask)
    image_ids.append(image_id)

# Convert to NumPy arrays for processing
predicted_masks = np.array(predicted_masks).reshape(-1, 1, orig_h, orig_w)
ground_truth_masks = np.array(ground_truth_masks).reshape(-1, 1, orig_h, orig_w)

# --------------------- Process & Save Results ---------------------
shape, index = predicted_masks.shape, image_ids
create_submission_file(predicted_masks, shape, index)  # Submission without Usage column
create_solution_file(ground_truth_masks, shape, index)  # Solution with ground truth masks

print("✅ **All tasks completed successfully!**")


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:
print("Solution records:", len(solution_data))  
print("Submission records:", len(submission_data))


In [ ]:
solution_df.dropna().to_csv(solution_csv_path, index=False)
submission_df.dropna().to_csv(submission_csv_path, index=False)


In [ ]:
print("Duplicate IDs in solution:", solution_df["id"].duplicated().sum())  
print("Duplicate IDs in submission:", submission_df["id"].duplicated().sum())


In [ ]:
print("Total mask files found:", len(mask_files))
print("Total test images found:", len(test_image_files))


In [ ]:
solution_df = solution_df.iloc[:1000]  # Keep only first 1000 rows
submission_df = submission_df.iloc[:1000]

solution_df.to_csv(solution_csv_path, index=False)
submission_df.to_csv(submission_csv_path, index=False)


In [ ]:
solution_df["ground_truth"] = solution_df["ground_truth"].apply(lambda x: f'"{x}"')
submission_df["predicted"] = submission_df["predicted"].apply(lambda x: f'"{x}"')


In [ ]:
solution_df.to_csv(solution_csv_path, index=False)
submission_df.to_csv(submission_csv_path, index=False)

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf

def run_length_encoding(mask):
    """Encodes a binary mask using Run-Length Encoding (RLE)."""
    pixels = mask.flatten(order='F')  # Flatten in column-major order (Fortran order)
    pixels = np.concatenate([[0], pixels, [0]])  # Add padding to detect transitions
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]  # Convert to (start, length) pairs
    
    return ' '.join(map(str, runs)) if len(runs) > 0 else ''  # Ensure correct format

def create_csv_file(image_ids, rle_masks, output_file, column_name, include_usage):
    """Creates a CSV file with image IDs and RLE masks. Adds 'Usage' only if needed."""
    data = {"id": image_ids, column_name: rle_masks}
    if include_usage:
        data["Usage"] = "Public"  # Only for solution.csv
    
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)
    print(f"CSV file '{output_file}' created successfully with {len(df)} records.")

# Load the trained model
model_path = "best_model.keras"
model = tf.keras.models.load_model(model_path)

# Define paths
test_images_path = "/kaggle/input/reduced-dataset/images/test"
ground_truth_masks_path = "/kaggle/input/reduced-dataset/masks/test"  # Modify if needed

# Get sorted list of test images
test_image_files = sorted([f for f in os.listdir(test_images_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

# Storage for predictions and ground truth
image_ids = []
submission_masks = []
solution_masks = []  # If ground truth is available

for image_file in test_image_files:
    image_id = os.path.splitext(image_file)[0]  # Extract image ID
    image_path = os.path.join(test_images_path, image_file)
    
    # Load and preprocess the test image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (256, 256))  # Resize to match model input shape
    image = image / 255.0  # Normalize
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    
    # Predict mask
    pred_mask = model.predict(image)[0]  # Remove batch dimension
    
    # Convert prediction to binary mask (threshold 0.5)
    binary_mask = (pred_mask > 0.5).astype(np.uint8)
    
    # Resize mask back to original image shape
    original_image = cv2.imread(image_path)  # Load original for size reference
    orig_h, orig_w = original_image.shape[:2]
    binary_mask = cv2.resize(binary_mask, (orig_w, orig_h), interpolation=cv2.INTER_NEAREST)
    
    # Encode prediction using RLE
    rle_pred = run_length_encoding(binary_mask)
    submission_masks.append(rle_pred)
    image_ids.append(image_id)

    # Load ground truth mask (if available)
    ground_truth_path = os.path.join(ground_truth_masks_path, image_file)
    if os.path.exists(ground_truth_path):
        gt_mask = cv2.imread(ground_truth_path, cv2.IMREAD_GRAYSCALE)  # Load in grayscale
        gt_mask = cv2.resize(gt_mask, (orig_w, orig_h), interpolation=cv2.INTER_NEAREST)
        gt_mask = (gt_mask > 0.5).astype(np.uint8)  # Convert to binary
        rle_gt = run_length_encoding(gt_mask)
        solution_masks.append(rle_gt)
    else:
        solution_masks.append("")  # Empty string if no ground truth mask

# Create submission (without "Usage") and solution (with "Usage") CSV files
create_csv_file(image_ids, submission_masks, "submissionf2.csv", "predicted", include_usage=False)
create_csv_file(image_ids, solution_masks, "solutionf2.csv", "ground_truth", include_usage=True)


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

def rle_encode(mask):
    '''
    mask: numpy array binary mask 
    1 - mask 
    0 - background
    Returns encoded run length 
    '''
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
# Define the path to the annotations folder for the test set
annotations_test_path = "/kaggle/input/reduced-dataset/annotations/test"

# List all mask files (assuming they are PNG images)
mask_files = sorted([f for f in os.listdir(annotations_test_path) if f.lower().endswith(".png")])
print("Total test masks found:", len(mask_files))

solution_data = []

for mask_file in mask_files:
    # Use the filename (without extension) as the unique id
    image_id = os.path.splitext(mask_file)[0]
    mask_path = os.path.join(annotations_test_path, mask_file)
    
    # Read the mask in grayscale
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"Warning: Could not read mask file {mask_path}")
        continue
    
    # Ensure the mask is binary (if needed, threshold it)
    # For example, assuming foreground > 0, convert to 0/1:
    mask = (mask > 0).astype(np.uint8)
    
    # Encode the binary mask using RLE
    rle_str = rle_encode(mask)
    
    solution_data.append({
        "id": image_id,
        "ground_truth": rle_str
    })

# Create a DataFrame from the solution data and save as CSV
solution_df = pd.DataFrame(solution_data)
solution_csv_path = "solutiongdf.csv"
solution_df["Usage"] = "Public"
solution_df.to_csv(solution_csv_path, index=False)

print("Solution CSV created successfully with", len(solution_df), "records.")
print("CSV saved at:", solution_csv_path)


In [ ]:
solution_df["Usage"] = "Public"
solution_csv_path = "solution1.csv"
solution_df.to_csv(solution_csv_path, index=False)